In [4]:
import pandas as pd
import numpy as np
import re

In [2]:
ori_twit_sexism_dataset = '/Users/churnika/Desktop/Projects/Crime_classification/dataset/twitter_sexism_parsed_dataset.csv'
ori_twit_sexism = pd.read_csv(ori_twit_sexism_dataset)
ori_twit_sexism = ori_twit_sexism.dropna()
ori_twit_sexism.columns

Index(['index', 'id', 'Text', 'Annotation', 'oh_label'], dtype='object')

In [3]:
ori_twit_sexism = ori_twit_sexism.drop(columns=['index','id','Annotation'])
print(ori_twit_sexism.columns)

Index(['Text', 'oh_label'], dtype='object')


In [9]:
msg_exp = ori_twit_sexism['Text'].str.lower()

In [19]:
print(msg_exp[785])

@jod0507 love yourself. 


In [12]:
# removing the urls that are present.

text_without_urls = []

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text_without_urls = url_pattern.sub('',text)
    return text_without_urls

for i in range(len(msg_exp)):
    try:
        msg_exp[i] = remove_urls(msg_exp[i])
        text_without_urls.append(msg_exp[i])
    except KeyError as e:
        # print(f"Error at index {i}: {e}. Skipping...")
        continue

In [18]:
print(text_without_urls[785])

@jod0507 love yourself. 


In [16]:
print(msg_exp[785])


@jod0507 love yourself. 
